# 알림 평가 에이전트

하드코딩된 규칙으로 알림 전송 가능 여부를 평가하는 에이전트
- 새벽 시간대 전송 금지 (예: 01:00 ~ 06:00)
- 알림 설정이 꺼져있으면 전송 금지

In [1]:
# 필요한 라이브러리 import
from datetime import datetime
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END

In [2]:
### State 정의

class NotificationEvaluationState(TypedDict):
    """알림 평가 에이전트의 State"""
    send_time: str  # 알림 전송 예정 시간 "YYYY-MM-DD HH:mm:ss"
    notification_enabled: bool  # 사용자 알림 설정 (True: 켜짐, False: 꺼짐)
    
    # 평가 결과
    is_allowed: Optional[bool]  # 알림 전송 허용 여부
    blocked_reasons: Optional[list]  # 차단된 이유 리스트
    evaluation_result: Optional[str]  # 평가 결과 설명

In [3]:
### 하드코딩된 규칙 정의

# 금지 시간대 (새벽 시간)
FORBIDDEN_HOURS = [1, 2, 3, 4, 5]  # 새벽 1시 ~ 5시

def is_forbidden_time(send_time_str: str) -> tuple[bool, str]:
    """
    전송 시간이 금지 시간대인지 확인
    
    Returns:
        (is_forbidden, reason)
    """
    try:
        send_time = datetime.fromisoformat(send_time_str)
        hour = send_time.hour
        
        if hour in FORBIDDEN_HOURS:
            return True, f"새벽 {hour}시는 알림 전송 금지 시간대입니다 (금지 시간: {FORBIDDEN_HOURS[0]}시 ~ {FORBIDDEN_HOURS[-1]}시)"
        
        return False, ""
    except Exception as e:
        return True, f"시간 파싱 실패: {e}"

def check_notification_setting(notification_enabled: bool) -> tuple[bool, str]:
    """
    알림 설정이 켜져있는지 확인
    
    Returns:
        (is_blocked, reason)
    """
    if not notification_enabled:
        return True, "사용자가 알림을 꺼놨습니다"
    
    return False, ""

In [4]:
### 노드 함수 정의

def evaluate_notification(state: NotificationEvaluationState) -> NotificationEvaluationState:
    """알림 전송 가능 여부를 하드코딩된 규칙으로 평가"""
    send_time = state["send_time"]
    notification_enabled = state["notification_enabled"]
    
    print(f"[evaluate_notification] 알림 평가 중...")
    print(f"  - 전송 예정 시간: {send_time}")
    print(f"  - 알림 설정: {'켜짐' if notification_enabled else '꺼짐'}")
    
    blocked_reasons = []
    
    # 규칙 1: 새벽 시간대 체크
    is_forbidden, time_reason = is_forbidden_time(send_time)
    if is_forbidden:
        blocked_reasons.append(time_reason)
        print(f"  ⛔ 시간대 차단: {time_reason}")
    
    # 규칙 2: 알림 설정 체크
    is_blocked, setting_reason = check_notification_setting(notification_enabled)
    if is_blocked:
        blocked_reasons.append(setting_reason)
        print(f"  ⛔ 설정 차단: {setting_reason}")
    
    # 최종 판단
    is_allowed = len(blocked_reasons) == 0
    
    if is_allowed:
        state["evaluation_result"] = "알림 전송이 허용되었습니다"
        print(f"  ✅ 알림 전송 허용")
    else:
        state["evaluation_result"] = f"알림 전송이 차단되었습니다: {', '.join(blocked_reasons)}"
        print(f"  ❌ 알림 전송 차단: {', '.join(blocked_reasons)}")
    
    state["is_allowed"] = is_allowed
    state["blocked_reasons"] = blocked_reasons
    
    return state

In [5]:
### 그래프 구성

# StateGraph 생성
workflow = StateGraph(NotificationEvaluationState)

# 노드 추가
workflow.add_node("evaluate_notification", evaluate_notification)

# 엣지 추가 - 단순 선형 흐름
workflow.set_entry_point("evaluate_notification")
workflow.add_edge("evaluate_notification", END)

# 그래프 컴파일
app = workflow.compile()

print("✅ 알림 평가 에이전트가 생성되었습니다!")
print("   - 하드코딩된 규칙으로 알림 전송 가능 여부를 평가합니다.")
print(f"   - 금지 시간대: 새벽 {FORBIDDEN_HOURS[0]}시 ~ {FORBIDDEN_HOURS[-1]}시")

✅ 알림 평가 에이전트가 생성되었습니다!
   - 하드코딩된 규칙으로 알림 전송 가능 여부를 평가합니다.
   - 금지 시간대: 새벽 1시 ~ 5시


In [6]:
### 평가 함수 (간편 사용을 위한 래퍼 함수)

def evaluate_notification_send(
    send_time: str,
    notification_enabled: bool = True
) -> dict:
    """
    알림 전송 가능 여부를 평가하는 간편 함수
    
    Args:
        send_time: 알림 전송 예정 시간 "YYYY-MM-DD HH:mm:ss"
        notification_enabled: 사용자 알림 설정 (기본값: True)
    
    Returns:
        평가 결과 딕셔너리
    """
    initial_state = NotificationEvaluationState(
        send_time=send_time,
        notification_enabled=notification_enabled,
        is_allowed=None,
        blocked_reasons=None,
        evaluation_result=None
    )
    
    result = app.invoke(initial_state)
    
    return {
        "is_allowed": result.get("is_allowed", False),
        "blocked_reasons": result.get("blocked_reasons", []),
        "evaluation_result": result.get("evaluation_result", "")
    }

In [7]:
### 테스트 실행

# ===== 테스트 케이스 1: 정상 시간대, 알림 켜짐 =====
print("=" * 60)
print("테스트 케이스 1: 정상 시간대, 알림 켜짐")
print("=" * 60)

result1 = evaluate_notification_send(
    send_time="2026-01-14 14:30:00",
    notification_enabled=True
)

print(f"\n전송 예정 시간: 2026-01-14 14:30:00")
print(f"알림 설정: 켜짐")
print(f"\n평가 결과: {result1['evaluation_result']}")
print(f"전송 허용: {result1['is_allowed']}")
if result1['blocked_reasons']:
    print(f"차단 사유: {', '.join(result1['blocked_reasons'])}")

# ===== 테스트 케이스 2: 새벽 시간대 (차단) =====
print("\n" + "=" * 60)
print("테스트 케이스 2: 새벽 시간대 (차단)")
print("=" * 60)

result2 = evaluate_notification_send(
    send_time="2026-01-14 01:30:00",  # 새벽 1시 30분
    notification_enabled=True
)

print(f"\n전송 예정 시간: 2026-01-14 01:30:00")
print(f"알림 설정: 켜짐")
print(f"\n평가 결과: {result2['evaluation_result']}")
print(f"전송 허용: {result2['is_allowed']}")
if result2['blocked_reasons']:
    print(f"차단 사유: {', '.join(result2['blocked_reasons'])}")

# ===== 테스트 케이스 3: 알림 꺼짐 (차단) =====
print("\n" + "=" * 60)
print("테스트 케이스 3: 알림 꺼짐 (차단)")
print("=" * 60)

result3 = evaluate_notification_send(
    send_time="2026-01-14 14:30:00",
    notification_enabled=False  # 알림 꺼짐
)

print(f"\n전송 예정 시간: 2026-01-14 14:30:00")
print(f"알림 설정: 꺼짐")
print(f"\n평가 결과: {result3['evaluation_result']}")
print(f"전송 허용: {result3['is_allowed']}")
if result3['blocked_reasons']:
    print(f"차단 사유: {', '.join(result3['blocked_reasons'])}")

# ===== 테스트 케이스 4: 새벽 시간대 + 알림 꺼짐 (이중 차단) =====
print("\n" + "=" * 60)
print("테스트 케이스 4: 새벽 시간대 + 알림 꺼짐 (이중 차단)")
print("=" * 60)

result4 = evaluate_notification_send(
    send_time="2026-01-14 03:00:00",  # 새벽 3시
    notification_enabled=False  # 알림 꺼짐
)

print(f"\n전송 예정 시간: 2026-01-14 03:00:00")
print(f"알림 설정: 꺼짐")
print(f"\n평가 결과: {result4['evaluation_result']}")
print(f"전송 허용: {result4['is_allowed']}")
if result4['blocked_reasons']:
    print(f"차단 사유:")
    for reason in result4['blocked_reasons']:
        print(f"  - {reason}")

print("\n" + "=" * 60)
print("✅ 알림 평가 완료!")
print("=" * 60)

테스트 케이스 1: 정상 시간대, 알림 켜짐
[evaluate_notification] 알림 평가 중...
  - 전송 예정 시간: 2026-01-14 14:30:00
  - 알림 설정: 켜짐
  ✅ 알림 전송 허용

전송 예정 시간: 2026-01-14 14:30:00
알림 설정: 켜짐

평가 결과: 알림 전송이 허용되었습니다
전송 허용: True

테스트 케이스 2: 새벽 시간대 (차단)
[evaluate_notification] 알림 평가 중...
  - 전송 예정 시간: 2026-01-14 01:30:00
  - 알림 설정: 켜짐
  ⛔ 시간대 차단: 새벽 1시는 알림 전송 금지 시간대입니다 (금지 시간: 1시 ~ 5시)
  ❌ 알림 전송 차단: 새벽 1시는 알림 전송 금지 시간대입니다 (금지 시간: 1시 ~ 5시)

전송 예정 시간: 2026-01-14 01:30:00
알림 설정: 켜짐

평가 결과: 알림 전송이 차단되었습니다: 새벽 1시는 알림 전송 금지 시간대입니다 (금지 시간: 1시 ~ 5시)
전송 허용: False
차단 사유: 새벽 1시는 알림 전송 금지 시간대입니다 (금지 시간: 1시 ~ 5시)

테스트 케이스 3: 알림 꺼짐 (차단)
[evaluate_notification] 알림 평가 중...
  - 전송 예정 시간: 2026-01-14 14:30:00
  - 알림 설정: 꺼짐
  ⛔ 설정 차단: 사용자가 알림을 꺼놨습니다
  ❌ 알림 전송 차단: 사용자가 알림을 꺼놨습니다

전송 예정 시간: 2026-01-14 14:30:00
알림 설정: 꺼짐

평가 결과: 알림 전송이 차단되었습니다: 사용자가 알림을 꺼놨습니다
전송 허용: False
차단 사유: 사용자가 알림을 꺼놨습니다

테스트 케이스 4: 새벽 시간대 + 알림 꺼짐 (이중 차단)
[evaluate_notification] 알림 평가 중...
  - 전송 예정 시간: 2026-01-14 03:00:00
  - 알림 설정: 꺼짐
  ⛔ 시간대 차단: 새벽 3시는 알림 전송 금지 시간대